# Image Classification: MNIST

Models:
+ [Vanilla NODE](#NODE)
+ [Aug.NODE](#ANODE)
+ [Input-Layer](#ILNODE)
+ [2nd-Ordered](#2NODE)

In [1]:
from torchdyn.models import *; from torchdyn import *
from torchdyn.nn import Augmenter

import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning.callbacks import Callback, ModelCheckpoint

from utils import Learner, get_MNIST_dloaders, MetricTracker

## Initialization

In [2]:
# quick run for automated notebook validation
dry_run = False

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cuda:0


## Data Loading

In [4]:
epoch = 20
path_to_data='./data/mnist_data'

trainloader, testloader = get_MNIST_dloaders(batch_size=64, size=28, path=path_to_data, download=False, num_workers=8)

<a id = 'ANODE'></a>
## Aug. Neural ODE

In [5]:
dim = 32
dim_half = 16
func = nn.Sequential(
        nn.GroupNorm(dim, dim),
        nn.Conv2d(dim , dim, 3, padding=1, bias=False),
        nn.Softplus(),
        nn.Conv2d(dim , dim, 3, padding=1, bias=False),
        nn.Softplus(),
        nn.GroupNorm(dim, dim),
        nn.Conv2d(dim , dim, 1),                     
    ).to(device)

In [6]:
nde = NeuralODE(func, 
               solver='dopri5',
               sensitivity='adjoint',
               atol=1e-4,
               rtol=1e-4,
               ).to(device)

# NOTE: the first noop `Augmenter` is used only to keep the `nde` at index `2`. Used to extract NFEs in Learner.
model = nn.Sequential(Augmenter(1, 0), # does nothing
                      Augmenter(1, 31),
                      nde,
                      nn.Conv2d(dim, 6, 1),
                      nn.AdaptiveAvgPool2d(4),
                      nn.Flatten(),                     
                      nn.Linear(6*16, 10)).to(device)

Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.


In [7]:
learn = Learner(model, trainloader, testloader, device)
cb2 = MetricTracker()
logger = TensorBoardLogger(save_dir='lightning_logs/MNIST/', name = 'model2')
trainer = pl.Trainer(max_epochs=epoch,
                     accelerator=accelerator,
                     fast_dev_run=dry_run,
                     gpus=torch.cuda.device_count(),
                     logger = logger,
                     callbacks = [cb2]
                     )
trainer.fit(learn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type             | Params
----------------------------------------------
0 | model    | Sequential       | 20.8 K
1 | loss     | CrossEntropyLoss | 0     
2 | accuarcy | Accuracy         | 0     
----------------------------------------------
20.8 K    Trainable params
0         Non-trainable params
20.8 K    Total params
0.083     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

C:\Users\m1052\miniconda3\envs\torchenv\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\result.py:229: UserWarning: You called `self.log('epoch', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6221824 bytes.

In [9]:
trainer.test(learn)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.9894000291824341
        test_loss           0.03209134191274643
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.03209134191274643, 'test_accuracy': 0.9894000291824341}]

In [10]:
file = './lightning_logs/MNIST/model2/logs.pt'
torch.save(cb2.collection, file)
t = torch.load(file)
t[-2:]

[{'epoch': tensor(11., device='cuda:0'),
  'train_loss': tensor(0.0226, device='cuda:0'),
  'accuracy': tensor(1., device='cuda:0'),
  'NFE': tensor(164., device='cuda:0')},
 {'test_loss': tensor(0.0321, device='cuda:0'),
  'test_accuracy': tensor(0.9894, device='cuda:0')}]